___
<img style="float: left; margin: 30px 15px 15px 15px;" src="https://upload.wikimedia.org/wikipedia/commons/d/db/Logo_ITESO_normal.jpg" width="250" height="320" /> 


# SEGUNDO EXAMEN PARCIAL
**MODELO NO LINEAL PARA PRONÓSTICOS**

## Examen Tema 2
### Nombre: José Manuel Haces López
*Fecha: 17 de abril del 2023*

*Por: Oscar David Jaramillo Z.*




___

In [ ]:
# Librerías
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Input, Flatten, Conv1D, MaxPooling1D, Bidirectional


# Cargando Utilidades creadas
import Utils

## Univariado

| 'H229', 'H251', 'H405', 'H136', 'H300' |        JOSE MANUEL HACES LOPEZ       |

### 0. Carga de Datos

In [ ]:
# Lista con las series de tiempo que se usarán
series_a_usar = sorted(['H229', 'H251', 'H405', 'H136', 'H300'])
print(f'Las series de tiempo a usar son: {series_a_usar}')

In [ ]:
# Cargamos la información de las series de tiempo
m4_info = pd.read_csv('./univariate/m4_info.csv')

# Filtramos las series de tiempo que se usarán
m4_info = m4_info[m4_info['M4id'].isin(series_a_usar)].reset_index(drop=True)

# Filtrando a las columnas que nos interesan
m4_info = m4_info[['M4id', 'SP', 'Frequency', 'StartingDate', 'Horizon']]

m4_info.head()

- Tenemos puras series de tiempo que se mueven cada hora (frecuencia 24 = Diario).
- No todas las series de tiempo tienen la misma fecha de inicio.
- En todas se busca hacer 48 horas (2 días) de forecasting.

In [ ]:
# Carga del archivo
univariado = pd.read_csv('./univariate/Hourly-train.csv')

# Filtando a solo las filas que nos interesan
univariado = univariado[univariado['V1'].isin(series_a_usar)].T.reset_index(drop=True)

# Poniendo las columnas adecuadas
univariado.columns = series_a_usar

# Eliminando la primera fila
univariado = univariado.drop(0).reset_index(drop=True)

# Visualización de los datos
univariado.head()

### 0.1) Creando las Series de Tiempo por Separado

In [ ]:
# Utilizando la función para separar las series de tiempo
Utils.separar_series(data_series = univariado, 
                     m4_info=m4_info, 
                     series_a_usar=series_a_usar, 
                     path='./univariate/Series_Tiempo/')

In [ ]:
# Cargando las series de tiempo
h136 = Utils.carga_serie('./univariate/Series_Tiempo/H136.csv')
h229 = Utils.carga_serie('./univariate/Series_Tiempo/H229.csv')
h251 = Utils.carga_serie('./univariate/Series_Tiempo/H251.csv')
h300 = Utils.carga_serie('./univariate/Series_Tiempo/H300.csv')
h405 = Utils.carga_serie('./univariate/Series_Tiempo/H405.csv')

## 1. Visualizaciones

In [ ]:
# Poniendo las series de tiempo en una lista
series = [h136, h229, h251, h300, h405]

# Usando la función para graficar las series de tiempo
Utils.plot_series(series, series_a_usar)

## 2. Modelado
____
### 2.1. Serie H136

In [ ]:
Utils.plot_series([h136], ['H136'])

#### 1. Preprocesamiento de los datos
- Al tener una serie que es bastante Estacional (que se repite mucho los patrones cada cierto tiempo), no veo necesario hacer un procesamiento de los datos, además de que parecen ser que son bastante simétricos.

In [ ]:
# Dandole formato a la serie de tiempo
X, y = Utils.split_univariate_sequence(sequence=h136, 
                                       column='H136', 
                                       n_steps=5, 
                                       tensor=False,
                                       )

- Se seleccionó *n_steps = 5*, porque tenemos una serie de tiempo con bajadas y subidas constantes así que seleccionamos un número mediano para poder saber si es una bajada prolongada o solo es una bajada y subida y poder aprender esa estacionalidad de la serie.

In [ ]:
# Separando los datos de entrenamiento y prueba
X_train, X_test, y_train, y_test = Utils.split_train_test(X=X, 
                                                          y=y, 
                                                          train_size=0.9)

#### 2. MLP - Multy Layer Perceptron
##### 2.1 MLP
- 2 Capas ocultas con 20 neuronas

In [ ]:
# Creando el modelo MLP con 5 capas ocultas de 50 neuronas
model_mlp1, h_mlp1 = Utils.gen_MLP_model(X=X_train, y=y_train, val_split=0.1, n_steps=5,
                                         activation='relu', num_layers=2, num_neurons=20,
                                         optimizer='Adam', lr=0.0001, loss='mse', metrics=['mae'],
                                         patience=20, epochs=500,  verbose=0,
                                         plot_history=True)


In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_mlp1 = Utils.plot_pred_test(nombre_modelo='MLP 1',
                                   title='MLP',
                                   pred=model_mlp1.predict(X_test),
                                   test=y_test
                                   )

##### 2.2 MLP
- 4 Capas ocultas con 30 neuronas

In [ ]:
# Creando el modelo MLP con 4 capas ocultas de 30 neuronas
model_mlp2, h_mlp2 = Utils.gen_MLP_model(X=X_train, y=y_train, val_split=0.1, n_steps=5,
                                         activation='relu', num_layers=4, num_neurons=30,
                                         optimizer='Adam', lr=0.003, loss='mse', metrics=['mae'],
                                         patience=20, epochs=500,  verbose=0,
                                         plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_mlp2 = Utils.plot_pred_test(nombre_modelo='MLP 2',
                                   title='MLP',
                                   pred=model_mlp2.predict(X_test),
                                   test=y_test
                                   )

##### 2.3 MLP
- 5 Capas ocultas con 50 neuronas

In [ ]:
# Creando el modelo MLP con 4 capas ocultas de 30 neuronas
model_mlp3, h_mlp3 = Utils.gen_MLP_model(X=X_train, y=y_train, val_split=0.1, n_steps=5,
                                         activation='relu', num_layers=5, num_neurons=50,
                                         optimizer='Adam', lr=0.003, loss='mse', metrics=['mae'],
                                         patience=20, epochs=500,  verbose=0,
                                         plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_mlp3 = Utils.plot_pred_test(nombre_modelo='MLP 3',
                                   title='MLP',
                                   pred=model_mlp3.predict(X_test),
                                   test=y_test
                                   )

**Conclusiones MLP**

In [ ]:
errores_MLP = Utils.concat_errores([errores_mlp1, errores_mlp2, errores_mlp3])
errores_MLP

In [ ]:
errores_mejor_MLP = Utils.plot_pred_test(nombre_modelo=errores_MLP['Modelo'][0],
                                         title=errores_MLP['Modelo'][0],
                                         pred=model_mlp2.predict(X_test),
                                         test=y_test)

#### 3. CNN - Convolutional Neural Network
##### 3.1. CNN 1
- Como se mencionó anteriormente, la serie tiene mucha estacionalidad muy marcada, por lo que una red CNN pequeña tendría que hacer el trabajo sin problema. Se seleccionó para empezar 2 capas convolucionales con 16 filtros de 2 y padding same, 5 capas ocultas con relu y 50 neuronas cada una.

In [ ]:
# Creando el modelo CNN con 2 capas CNN de 16 filtros y 5 capas Dense de 50 neuronas
model_cnn, h_cnn = Utils.gen_CNN_model(X=X_train, y=y_train, val_split=0.1, n_steps=5,
                                       num_layers_cnn=2, num_filters=16, kernel_size=2, padding='same',
                                       activation='relu', num_layers_dense=5, num_neurons=50,
                                       optimizer='Adam', lr=0.003, loss='mse', metrics=['mae'],
                                       patience=20, epochs=500,  verbose=0,
                                       plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_cnn = Utils.plot_pred_test(nombre_modelo='CNN 1',
                                   title='CNN 1',
                                   pred=model_cnn.predict(X_test),
                                   test=y_test
                                   )

##### 3.2. CNN 2
- Agregando 2 capas convolucionales, quitando filtros y haciendo más grande el tamaño de los filtros. Buscamos que los filtros hagan un mejor trabajo.
- ASí mismo disminuimos el número de densasa para ver si funciona.

In [ ]:
# Creando el modelo CNN con 2 capas CNN de 16 filtros y 5 capas Dense de 50 neuronas
model_cnn2, h_cnn2 = Utils.gen_CNN_model(X=X_train, y=y_train, val_split=0.1, n_steps=5,
                                       num_layers_cnn=4, num_filters=4, kernel_size=3, padding='same',
                                       activation='relu', num_layers_dense=2, num_neurons=20,
                                       optimizer='Adam', lr=0.003, loss='mse', metrics=['mae'],
                                       patience=20, epochs=500,  verbose=0,
                                       plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_cnn2 = Utils.plot_pred_test(nombre_modelo='CNN2',
                                   title='CNN 2',
                                   pred=model_cnn2.predict(X_test),
                                   test=y_test
                                   )

##### 3.3. CNN 3
- Parece ser que si funciona, aumentaremos un poco más la parte convolucioneal y también aumentaremos la parte de predicción.
- 10 capas convolucionales con 8 filtros y tamaño de 5.
- 5 capas ocultas y 50 neuronas
- Bajando el LR para bajar en la cola un poco más e intentar llegar un poco más abajo en la función de perdida.

In [ ]:
# Creando el modelo CNN con 2 capas CNN de 16 filtros y 5 capas Dense de 50 neuronas
model_cnn3, h_cnn3 = Utils.gen_CNN_model(X=X_train, y=y_train, val_split=0.1, n_steps=5,
                                       num_layers_cnn=10, num_filters=8, kernel_size=5, padding='same',
                                       activation='relu', num_layers_dense=5, num_neurons=50,
                                       optimizer='Adam', lr=0.0003, loss='mse', metrics=['mae'],
                                       patience=20, epochs=500,  verbose=0,
                                       plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_cnn3 = Utils.plot_pred_test(nombre_modelo='CNN 3',
                                   title='CNN 3',
                                   pred=model_cnn2.predict(X_test),
                                   test=y_test
                                   )

**Conclusiones CNN**

In [ ]:
errores_CNN = Utils.concat_errores([errores_cnn, errores_cnn2, errores_cnn3])
errores_CNN

In [ ]:
errores_mejor_CNN = Utils.plot_pred_test(nombre_modelo=errores_CNN['Modelo'][0],
                                         title=errores_CNN['Modelo'][0],
                                         pred=model_cnn.predict(X_test),
                                         test=y_test)

- Podemos ver que el aumento de la parte convolucional, sin embargo al aumentarla todavía más no logramos mejorar el rendimiento, por lo que seleccioné el 2 ya que es menos complejo.

#### 4. LSTM
- Para poder usar LSTM tenemos que convertir nuestros datos a tensor, por lo que rehacemos los split y Train y test.

In [ ]:
# Separando los datos de entrenamiento y prueba y haciendolo tensor
X_tens, y_tens = Utils.split_univariate_sequence(sequence=h136,
                                                 column='H136',
                                                 n_steps=5,
                                                 tensor=True,
                                                 n_features=1)

In [ ]:
# Dividiendo los datos en train y test
X_train_tens, X_test_tens, y_train_tens, y_test_tens = Utils.split_train_test(X=X_tens,
                                                                    y=y_tens,
                                                                    train_size=0.9
                                                                    )

##### 4.1 LSTM

In [ ]:
# Creando el modelo LSTM con 2 capas LSTM de 50 neuronas y 5 capas Dense de 50 neuronas
model_lstm, h_lstm = Utils.gen_LSTM_model(X=X_train_tens, y=y_train_tens, val_split=0.1, n_steps=5, n_features=1,
                                          num_layers_lstm=2, activation_lstm='tanh', num_units_lstm=50, bidireccional=False,
                                          activation='relu', num_layers_dense=5, num_neurons=50,
                                          optimizer='Adam', lr=0.0003, loss='mse', metrics=['mae'],
                                          patience=20, epochs=500, verbose=0,
                                          plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_lstm = Utils.plot_pred_test(nombre_modelo='LSTM 1',
                                   title='LSTM 1',
                                   pred=model_lstm.predict(X_test_tens),
                                   test=y_test
                                   )

##### 4.2 LSTM
- Aumentando capas de LSTM para ver si mejora
- 4 capas densas en vez de 2

In [ ]:
# Creando el modelo LSTM con 4 capas LSTM de 50 neuronas y 5 capas Dense de 50 neuronas
model_lstm2, h_lstm2 = Utils.gen_LSTM_model(X=X_train_tens, y=y_train_tens, val_split=0.1, n_steps=5, n_features=1,
                                          num_layers_lstm=4, activation_lstm='tanh', num_units_lstm=50, bidireccional=False,
                                          activation='relu', num_layers_dense=5, num_neurons=50,
                                          optimizer='Adam', lr=0.0003, loss='mse', metrics=['mae'],
                                          patience=20, epochs=500, verbose=0,
                                          plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_lstm2 = Utils.plot_pred_test(nombre_modelo='LSTM 2',
                                   title='LSTM 2',
                                   pred=model_lstm2.predict(X_test_tens),
                                   test=y_test
                                   )

##### 4.3 LSTM

In [ ]:
# Creando el modelo LSTM con 4 capas LSTM de 50 neuronas y 5 capas Dense de 50 neuronas
model_lstm3, h_lstm3 = Utils.gen_LSTM_model(X=X_train_tens, y=y_train_tens, val_split=0.1, n_steps=5, n_features=1,
                                          num_layers_lstm=5, activation_lstm='tanh', num_units_lstm=50, bidireccional=True,
                                          activation='relu', num_layers_dense=5, num_neurons=50,
                                          optimizer='Adam', lr=0.0003, loss='mse', metrics=['mae'],
                                          patience=20, epochs=500, verbose=0,
                                          plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_lstm3 = Utils.plot_pred_test(nombre_modelo='LSTM 3',
                                   title='LSTM 3',
                                   pred=model_lstm3.predict(X_test_tens),
                                   test=y_test
                                   )

**Conclusiones**

In [ ]:
errores_LSTM = Utils.concat_errores([errores_lstm, errores_lstm2, errores_lstm3])
errores_LSTM

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_mejor_lstm = Utils.plot_pred_test(nombre_modelo=errores_LSTM['Modelo'][0],
                                  title=errores_LSTM['Modelo'][0],
                                  pred=model_lstm.predict(X_test_tens),
                                  test=y_test)

- Tenemos un error casi nulo, sorprendemente mientras más aumentabamos las capas o la cantidad de unidades por cada (como en el caso 1 y 2 no bajo el error, parece ser que es un muy buen modelo)

#### 5. CNN - LSTM
##### 5.1. CNN - LSTM

In [ ]:
# Separando los datos de entrenamiento y prueba y haciendolo tensor con 6 pasos
X_tens2, y_tens2 = Utils.split_univariate_sequence(sequence=h136,
                                                 column='H136',
                                                 n_steps=6,
                                                 tensor=True,
                                                 n_features=1)

# Dividiendo los datos en train y test
X_train_tens2, X_test_tens2, y_train_tens2, y_test_tens2 = Utils.split_train_test(X=X_tens2,
                                                                                  y=y_tens2,
                                                                                  train_size=0.9
                                                                                    )

# Cambiando el shape del X_train_tens y X_test_tens
n_seq = 2
n_steps = 3
n_features = 1

# Cambiando el shape del X_train_tens y X_test_tens
X_train_tens2 = X_train_tens2.reshape((X_train_tens2.shape[0], n_seq, n_steps, n_features))
X_test_tens2 = X_test_tens2.reshape((X_test_tens2.shape[0], n_seq, n_steps, n_features))

In [ ]:
model_cnn_lstm1, h1_cnn_lstm = Utils.gen_CNN_LSTM_model(X=X_train_tens2, y=y_train_tens2, val_split=0.1, n_steps=5, n_features=1,
                                                  num_layers_cnn=4, num_filters=16, kernel_size=3, padding='same',
                                                  num_layers_lstm=5, activation_lstm='tanh', num_units_lstm=50,
                                                  activation='relu', num_layers_dense=5, num_neurons=50, 
                                                  optimizer='Adam', lr=0.0003, loss='mse', metrics=['mae'],
                                                  patience=20, epochs=500, verbose=0,
                                                  plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_cnn_lstm1 = Utils.plot_pred_test(nombre_modelo='CNN - LSTM 1',
                                          title='CNN - LSTM 1',
                                          pred=model_cnn_lstm1.predict(X_test_tens2),
                                          test=y_test_tens2
                                          )

##### 5.2 CNN - LSTM 2

In [ ]:
model_cnn_lstm2, h2_cnn_lstm = Utils.gen_CNN_LSTM_model(X=X_train_tens2, y=y_train_tens2, val_split=0.1, n_steps=5, n_features=1,
                                                  num_layers_cnn=2, num_filters=32, kernel_size=3, padding='same',
                                                  num_layers_lstm=5, activation_lstm='tanh', num_units_lstm=50,
                                                  activation='relu', num_layers_dense=5, num_neurons=50, 
                                                  optimizer='Adam', lr=0.0003, loss='mse', metrics=['mae'],
                                                  patience=20, epochs=500, verbose=0,
                                                  plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_cnn_lstm2 = Utils.plot_pred_test(nombre_modelo='CNN - LSTM 2',
                                          title='CNN - LSTM 2',
                                          pred=model_cnn_lstm2.predict(X_test_tens2),
                                          test=y_test_tens2
                                          )

##### 5.3 CNN - LSTM 3

In [ ]:
model_cnn_lstm3, h3_cnn_lstm = Utils.gen_CNN_LSTM_model(X=X_train_tens2, y=y_train_tens2, val_split=0.1, n_steps=5, n_features=1,
                                                  num_layers_cnn=8, num_filters=4, kernel_size=4, padding='same',
                                                  num_layers_lstm=5, activation_lstm='tanh', num_units_lstm=50,
                                                  activation='relu', num_layers_dense=5, num_neurons=50, 
                                                  optimizer='Adam', lr=0.0003, loss='mse', metrics=['mae'],
                                                  patience=20, epochs=500, verbose=0,
                                                  plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_cnn_lstm3 = Utils.plot_pred_test(nombre_modelo='CNN - LSTM 3',
                                          title='CNN - LSTM 3',
                                          pred=model_cnn_lstm3.predict(X_test_tens2),
                                          test=y_test_tens2
                                          )

**Conclusiones**

In [ ]:
errores_cnn_lstm = Utils.concat_errores([errores_cnn_lstm1, errores_cnn_lstm2, errores_cnn_lstm3])
errores_cnn_lstm

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_mejor_CNN_LSTM = Utils.plot_pred_test(nombre_modelo=errores_cnn_lstm['Modelo'][0],
                                  title=errores_cnn_lstm['Modelo'][0],
                                  pred=model_cnn_lstm1.predict(X_test_tens2),
                                  test=y_test)

**Sacando el mejor modelo**

In [ ]:
mejores_modelos = Utils.concat_errores([errores_mejor_MLP, errores_mejor_CNN, errores_mejor_lstm, errores_mejor_CNN_LSTM]).sort_values(by='MAE')
mejores_modelos

- El mejor modelo es el LSTM, por lo que es el que haremos la predicción

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_mejor_lstm = Utils.plot_pred_test(nombre_modelo=errores_LSTM['Modelo'][0],
                                  title=errores_LSTM['Modelo'][0],
                                  pred=model_lstm.predict(X_test_tens[:48]),
                                  test=y_test[:48])

- No se hizo ajuste de hiperparámetros ya que tenemos un r2 de 0.99, lo cual buscar mejorarlo es overfitearlo y desperdiciar poder de computo.

____
### 2.2. Serie H229

In [ ]:
Utils.plot_series([h229], ['H229'])

#### 1. Preprocesamiento de los datos
- Al tener una serie que es bastante Estacional (que se repite mucho los patrones cada cierto tiempo), no veo necesario hacer un procesamiento de los datos, además de que parecen ser que son bastante simétricos.

In [ ]:
# Dandole formato a la serie de tiempo
X, y = Utils.split_univariate_sequence(sequence=h229, 
                                       column='H229', 
                                       n_steps=5, 
                                       tensor=False,
                                       )

- Se seleccionó *n_steps = 5*, porque tenemos una serie de tiempo con bajadas y subidas constantes así que seleccionamos un número mediano para poder saber si es una bajada prolongada o solo es una bajada y subida y poder aprender esa estacionalidad de la serie.

In [ ]:
# Separando los datos de entrenamiento y prueba
X_train, X_test, y_train, y_test = Utils.split_train_test(X=X, 
                                                          y=y, 
                                                          train_size=0.9)

#### 2. MLP - Multy Layer Perceptron
##### 2.1 MLP
- 2 Capas ocultas con 20 neuronas

In [ ]:
# Creando el modelo MLP con 5 capas ocultas de 50 neuronas
model_mlp1, h_mlp1 = Utils.gen_MLP_model(X=X_train, y=y_train, val_split=0.1, n_steps=5,
                                         activation='relu', num_layers=2, num_neurons=20,
                                         optimizer='Adam', lr=0.0001, loss='mse', metrics=['mae'],
                                         patience=20, epochs=500,  verbose=0,
                                         plot_history=True)


In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_mlp1 = Utils.plot_pred_test(nombre_modelo='MLP 1',
                                   title='MLP',
                                   pred=model_mlp1.predict(X_test),
                                   test=y_test
                                   )

##### 2.2 MLP
- 4 Capas ocultas con 30 neuronas

In [ ]:
# Creando el modelo MLP con 4 capas ocultas de 30 neuronas
model_mlp2, h_mlp2 = Utils.gen_MLP_model(X=X_train, y=y_train, val_split=0.1, n_steps=5,
                                         activation='relu', num_layers=4, num_neurons=30,
                                         optimizer='Adam', lr=0.003, loss='mse', metrics=['mae'],
                                         patience=20, epochs=500,  verbose=0,
                                         plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_mlp2 = Utils.plot_pred_test(nombre_modelo='MLP 2',
                                   title='MLP',
                                   pred=model_mlp2.predict(X_test),
                                   test=y_test
                                   )

##### 2.3 MLP
- 5 Capas ocultas con 50 neuronas

In [ ]:
# Creando el modelo MLP con 4 capas ocultas de 30 neuronas
model_mlp3, h_mlp3 = Utils.gen_MLP_model(X=X_train, y=y_train, val_split=0.1, n_steps=5,
                                         activation='relu', num_layers=5, num_neurons=50,
                                         optimizer='Adam', lr=0.003, loss='mse', metrics=['mae'],
                                         patience=20, epochs=500,  verbose=0,
                                         plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_mlp3 = Utils.plot_pred_test(nombre_modelo='MLP 3',
                                   title='MLP',
                                   pred=model_mlp3.predict(X_test),
                                   test=y_test
                                   )

**Conclusiones MLP**

In [ ]:
errores_MLP = Utils.concat_errores([errores_mlp1, errores_mlp2, errores_mlp3])
errores_MLP

In [ ]:
errores_mejor_MLP = Utils.plot_pred_test(nombre_modelo=errores_MLP['Modelo'][0],
                                         title=errores_MLP['Modelo'][0],
                                         pred=model_mlp2.predict(X_test),
                                         test=y_test)

#### 3. CNN - Convolutional Neural Network
##### 3.1. CNN 1
- Como se mencionó anteriormente, la serie tiene mucha estacionalidad muy marcada, por lo que una red CNN pequeña tendría que hacer el trabajo sin problema. Se seleccionó para empezar 2 capas convolucionales con 16 filtros de 2 y padding same, 5 capas ocultas con relu y 50 neuronas cada una.

In [ ]:
# Creando el modelo CNN con 2 capas CNN de 16 filtros y 5 capas Dense de 50 neuronas
model_cnn, h_cnn = Utils.gen_CNN_model(X=X_train, y=y_train, val_split=0.1, n_steps=5,
                                       num_layers_cnn=2, num_filters=16, kernel_size=2, padding='same',
                                       activation='relu', num_layers_dense=5, num_neurons=50,
                                       optimizer='Adam', lr=0.003, loss='mse', metrics=['mae'],
                                       patience=20, epochs=500,  verbose=0,
                                       plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_cnn = Utils.plot_pred_test(nombre_modelo='CNN 1',
                                   title='CNN 1',
                                   pred=model_cnn.predict(X_test),
                                   test=y_test
                                   )

##### 3.2. CNN 2
- Agregando 2 capas convolucionales, quitando filtros y haciendo más grande el tamaño de los filtros. Buscamos que los filtros hagan un mejor trabajo.
- ASí mismo disminuimos el número de densasa para ver si funciona.

In [ ]:
# Creando el modelo CNN con 2 capas CNN de 16 filtros y 5 capas Dense de 50 neuronas
model_cnn2, h_cnn2 = Utils.gen_CNN_model(X=X_train, y=y_train, val_split=0.1, n_steps=5,
                                       num_layers_cnn=4, num_filters=4, kernel_size=3, padding='same',
                                       activation='relu', num_layers_dense=2, num_neurons=20,
                                       optimizer='Adam', lr=0.003, loss='mse', metrics=['mae'],
                                       patience=20, epochs=500,  verbose=0,
                                       plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_cnn2 = Utils.plot_pred_test(nombre_modelo='CNN2',
                                   title='CNN 2',
                                   pred=model_cnn2.predict(X_test),
                                   test=y_test
                                   )

##### 3.3. CNN 3
- Parece ser que si funciona, aumentaremos un poco más la parte convolucioneal y también aumentaremos la parte de predicción.
- 10 capas convolucionales con 8 filtros y tamaño de 5.
- 5 capas ocultas y 50 neuronas
- Bajando el LR para bajar en la cola un poco más e intentar llegar un poco más abajo en la función de perdida.

In [ ]:
# Creando el modelo CNN con 2 capas CNN de 16 filtros y 5 capas Dense de 50 neuronas
model_cnn3, h_cnn3 = Utils.gen_CNN_model(X=X_train, y=y_train, val_split=0.1, n_steps=5,
                                       num_layers_cnn=10, num_filters=8, kernel_size=5, padding='same',
                                       activation='relu', num_layers_dense=5, num_neurons=50,
                                       optimizer='Adam', lr=0.0003, loss='mse', metrics=['mae'],
                                       patience=20, epochs=500,  verbose=0,
                                       plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_cnn3 = Utils.plot_pred_test(nombre_modelo='CNN 3',
                                   title='CNN 3',
                                   pred=model_cnn2.predict(X_test),
                                   test=y_test
                                   )

**Conclusiones CNN**

In [ ]:
errores_CNN = Utils.concat_errores([errores_cnn, errores_cnn2, errores_cnn3])
errores_CNN

In [ ]:
errores_mejor_CNN = Utils.plot_pred_test(nombre_modelo=errores_CNN['Modelo'][0],
                                         title=errores_CNN['Modelo'][0],
                                         pred=model_cnn.predict(X_test),
                                         test=y_test)

- Podemos ver que el aumento de la parte convolucional, sin embargo al aumentarla todavía más no logramos mejorar el rendimiento, por lo que seleccioné el 2 ya que es menos complejo.

#### 4. LSTM
- Para poder usar LSTM tenemos que convertir nuestros datos a tensor, por lo que rehacemos los split y Train y test.

In [ ]:
# Separando los datos de entrenamiento y prueba y haciendolo tensor
X_tens, y_tens = Utils.split_univariate_sequence(sequence=h229,
                                                 column='H229',
                                                 n_steps=5,
                                                 tensor=True,
                                                 n_features=1)

In [ ]:
# Dividiendo los datos en train y test
X_train_tens, X_test_tens, y_train_tens, y_test_tens = Utils.split_train_test(X=X_tens,
                                                                    y=y_tens,
                                                                    train_size=0.9
                                                                    )

##### 4.1 LSTM

In [ ]:
# Creando el modelo LSTM con 2 capas LSTM de 50 neuronas y 5 capas Dense de 50 neuronas
model_lstm, h_lstm = Utils.gen_LSTM_model(X=X_train_tens, y=y_train_tens, val_split=0.1, n_steps=5, n_features=1,
                                          num_layers_lstm=2, activation_lstm='tanh', num_units_lstm=50, bidireccional=False,
                                          activation='relu', num_layers_dense=5, num_neurons=50,
                                          optimizer='Adam', lr=0.0003, loss='mse', metrics=['mae'],
                                          patience=20, epochs=500, verbose=0,
                                          plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_lstm = Utils.plot_pred_test(nombre_modelo='LSTM 1',
                                   title='LSTM 1',
                                   pred=model_lstm.predict(X_test_tens),
                                   test=y_test
                                   )

##### 4.2 LSTM
- Aumentando capas de LSTM para ver si mejora
- 4 capas densas en vez de 2

In [ ]:
# Creando el modelo LSTM con 4 capas LSTM de 50 neuronas y 5 capas Dense de 50 neuronas
model_lstm2, h_lstm2 = Utils.gen_LSTM_model(X=X_train_tens, y=y_train_tens, val_split=0.1, n_steps=5, n_features=1,
                                          num_layers_lstm=4, activation_lstm='tanh', num_units_lstm=50, bidireccional=False,
                                          activation='relu', num_layers_dense=5, num_neurons=50,
                                          optimizer='Adam', lr=0.0003, loss='mse', metrics=['mae'],
                                          patience=20, epochs=500, verbose=0,
                                          plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_lstm2 = Utils.plot_pred_test(nombre_modelo='LSTM 2',
                                   title='LSTM 2',
                                   pred=model_lstm2.predict(X_test_tens),
                                   test=y_test
                                   )

##### 4.3 LSTM

In [ ]:
# Creando el modelo LSTM con 4 capas LSTM de 50 neuronas y 5 capas Dense de 50 neuronas
model_lstm3, h_lstm3 = Utils.gen_LSTM_model(X=X_train_tens, y=y_train_tens, val_split=0.1, n_steps=5, n_features=1,
                                          num_layers_lstm=5, activation_lstm='tanh', num_units_lstm=50, bidireccional=True,
                                          activation='relu', num_layers_dense=5, num_neurons=50,
                                          optimizer='Adam', lr=0.0003, loss='mse', metrics=['mae'],
                                          patience=20, epochs=500, verbose=0,
                                          plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_lstm3 = Utils.plot_pred_test(nombre_modelo='LSTM 3',
                                   title='LSTM 3',
                                   pred=model_lstm3.predict(X_test_tens),
                                   test=y_test
                                   )

**Conclusiones**

In [ ]:
errores_LSTM = Utils.concat_errores([errores_lstm, errores_lstm2, errores_lstm3])
errores_LSTM

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
mejor_lstm = Utils.plot_pred_test(nombre_modelo=errores_LSTM['Modelo'][0],
                                  title=errores_LSTM['Modelo'][0],
                                  pred=model_lstm.predict(X_test_tens),
                                  test=y_test)

- Tenemos un error casi nulo, sorprendemente mientras más aumentabamos las capas o la cantidad de unidades por cada (como en el caso 1 y 2 no bajo el error, parece ser que es un muy buen modelo)

#### 5. CNN - LSTM
##### 5.1. CNN - LSTM

In [ ]:
# Separando los datos de entrenamiento y prueba y haciendolo tensor con 6 pasos
X_tens2, y_tens2 = Utils.split_univariate_sequence(sequence=h229,
                                                 column='H229',
                                                 n_steps=6,
                                                 tensor=True,
                                                 n_features=1)

# Dividiendo los datos en train y test
X_train_tens2, X_test_tens2, y_train_tens2, y_test_tens2 = Utils.split_train_test(X=X_tens2,
                                                                                  y=y_tens2,
                                                                                  train_size=0.9
                                                                                    )

# Cambiando el shape del X_train_tens y X_test_tens
n_seq = 2
n_steps = 3
n_features = 1

# Cambiando el shape del X_train_tens y X_test_tens
X_train_tens2 = X_train_tens2.reshape((X_train_tens2.shape[0], n_seq, n_steps, n_features))
X_test_tens2 = X_test_tens2.reshape((X_test_tens2.shape[0], n_seq, n_steps, n_features))

In [ ]:
model_cnn_lstm1, h1_cnn_lstm = Utils.gen_CNN_LSTM_model(X=X_train_tens2, y=y_train_tens2, val_split=0.1, n_steps=5, n_features=1,
                                                  num_layers_cnn=4, num_filters=16, kernel_size=3, padding='same',
                                                  num_layers_lstm=5, activation_lstm='tanh', num_units_lstm=50,
                                                  activation='relu', num_layers_dense=5, num_neurons=50, 
                                                  optimizer='Adam', lr=0.0003, loss='mse', metrics=['mae'],
                                                  patience=20, epochs=500, verbose=0,
                                                  plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_cnn_lstm1 = Utils.plot_pred_test(nombre_modelo='CNN - LSTM 1',
                                          title='CNN - LSTM 1',
                                          pred=model_cnn_lstm1.predict(X_test_tens2),
                                          test=y_test_tens2
                                          )

##### 5.2 CNN - LSTM 2

In [ ]:
model_cnn_lstm2, h2_cnn_lstm = Utils.gen_CNN_LSTM_model(X=X_train_tens2, y=y_train_tens2, val_split=0.1, n_steps=5, n_features=1,
                                                  num_layers_cnn=2, num_filters=32, kernel_size=3, padding='same',
                                                  num_layers_lstm=5, activation_lstm='tanh', num_units_lstm=50,
                                                  activation='relu', num_layers_dense=5, num_neurons=50, 
                                                  optimizer='Adam', lr=0.0003, loss='mse', metrics=['mae'],
                                                  patience=20, epochs=500, verbose=0,
                                                  plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_cnn_lstm1 = Utils.plot_pred_test(nombre_modelo='CNN - LSTM 2',
                                          title='CNN - LSTM 2',
                                          pred=model_cnn_lstm2.predict(X_test_tens2),
                                          test=y_test_tens2
                                          )

##### 5.3 CNN - LSTM 3

In [ ]:
model_cnn_lstm3, h3_cnn_lstm = Utils.gen_CNN_LSTM_model(X=X_train_tens2, y=y_train_tens2, val_split=0.1, n_steps=5, n_features=1,
                                                  num_layers_cnn=8, num_filters=4, kernel_size=4, padding='same',
                                                  num_layers_lstm=5, activation_lstm='tanh', num_units_lstm=50,
                                                  activation='relu', num_layers_dense=5, num_neurons=50, 
                                                  optimizer='Adam', lr=0.0003, loss='mse', metrics=['mae'],
                                                  patience=20, epochs=500, verbose=0,
                                                  plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_cnn_lstm1 = Utils.plot_pred_test(nombre_modelo='CNN - LSTM 3',
                                          title='CNN - LSTM 3',
                                          pred=model_cnn_lstm3.predict(X_test_tens2),
                                          test=y_test_tens2
                                          )

**Conclusiones**

In [ ]:
errores_cnn_lstm = Utils.concat_errores([errores_cnn_lstm1, errores_cnn_lstm2, errores_cnn_lstm3])
errores_cnn_lstm

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_mejor_CNN_LSTM = Utils.plot_pred_test(nombre_modelo=errores_cnn_lstm['Modelo'][0],
                                  title=errores_cnn_lstm['Modelo'][0],
                                  pred=model_cnn_lstm1.predict(X_test_tens2),
                                  test=y_test)

**Sacando el mejor modelo**

In [ ]:
mejores_modelos = Utils.concat_errores([errores_mejor_MLP, errores_mejor_CNN, errores_mejor_lstm, errores_mejor_CNN_LSTM]).sort_values(by='MAE')
mejores_modelos

- El mejor modelo es el LSTM, por lo que es el que haremos la predicción

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_mejor_lstm = Utils.plot_pred_test(nombre_modelo=errores_LSTM['Modelo'][0],
                                  title=errores_LSTM['Modelo'][0],
                                  pred=model_lstm.predict(X_test_tens[:48]),
                                  test=y_test[:48])

- No se hizo ajuste de hiperparámetros ya que tenemos un r2 de 0.99, lo cual buscar mejorarlo es overfitearlo y desperdiciar poder de computo.

____
### 2.3. Serie H251

In [ ]:
Utils.plot_series([h251], ['H251'])

#### 1. Preprocesamiento de los datos
- Al tener una serie que es bastante Estacional (que se repite mucho los patrones cada cierto tiempo), no veo necesario hacer un procesamiento de los datos, además de que parecen ser que son bastante simétricos.

In [ ]:
# Dandole formato a la serie de tiempo
X, y = Utils.split_univariate_sequence(sequence=h251, 
                                       column='H251', 
                                       n_steps=5, 
                                       tensor=False,
                                       )

- Se seleccionó *n_steps = 5*, porque tenemos una serie de tiempo con bajadas y subidas constantes así que seleccionamos un número mediano para poder saber si es una bajada prolongada o solo es una bajada y subida y poder aprender esa estacionalidad de la serie.

In [ ]:
# Separando los datos de entrenamiento y prueba
X_train, X_test, y_train, y_test = Utils.split_train_test(X=X, 
                                                          y=y, 
                                                          train_size=0.9)

#### 2. MLP - Multy Layer Perceptron
##### 2.1 MLP
- 2 Capas ocultas con 20 neuronas

In [ ]:
# Creando el modelo MLP con 5 capas ocultas de 50 neuronas
model_mlp1, h_mlp1 = Utils.gen_MLP_model(X=X_train, y=y_train, val_split=0.1, n_steps=5,
                                         activation='relu', num_layers=2, num_neurons=20,
                                         optimizer='Adam', lr=0.0001, loss='mse', metrics=['mae'],
                                         patience=20, epochs=500,  verbose=0,
                                         plot_history=True)


In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_mlp1 = Utils.plot_pred_test(nombre_modelo='MLP 1',
                                   title='MLP',
                                   pred=model_mlp1.predict(X_test),
                                   test=y_test
                                   )

##### 2.2 MLP
- 4 Capas ocultas con 30 neuronas

In [ ]:
# Creando el modelo MLP con 4 capas ocultas de 30 neuronas
model_mlp2, h_mlp2 = Utils.gen_MLP_model(X=X_train, y=y_train, val_split=0.1, n_steps=5,
                                         activation='relu', num_layers=4, num_neurons=30,
                                         optimizer='Adam', lr=0.003, loss='mse', metrics=['mae'],
                                         patience=20, epochs=500,  verbose=0,
                                         plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_mlp2 = Utils.plot_pred_test(nombre_modelo='MLP 2',
                                   title='MLP',
                                   pred=model_mlp2.predict(X_test),
                                   test=y_test
                                   )

##### 2.3 MLP
- 5 Capas ocultas con 50 neuronas

In [ ]:
# Creando el modelo MLP con 4 capas ocultas de 30 neuronas
model_mlp3, h_mlp3 = Utils.gen_MLP_model(X=X_train, y=y_train, val_split=0.1, n_steps=5,
                                         activation='relu', num_layers=5, num_neurons=50,
                                         optimizer='Adam', lr=0.003, loss='mse', metrics=['mae'],
                                         patience=20, epochs=500,  verbose=0,
                                         plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_mlp3 = Utils.plot_pred_test(nombre_modelo='MLP 3',
                                   title='MLP',
                                   pred=model_mlp3.predict(X_test),
                                   test=y_test
                                   )

**Conclusiones MLP**

In [ ]:
errores_MLP = Utils.concat_errores([errores_mlp1, errores_mlp2, errores_mlp3])
errores_MLP

In [ ]:
errores_mejor_MLP = Utils.plot_pred_test(nombre_modelo=errores_MLP['Modelo'][0],
                                         title=errores_MLP['Modelo'][0],
                                         pred=model_mlp3.predict(X_test),
                                         test=y_test)

#### 3. CNN - Convolutional Neural Network
##### 3.1. CNN 1
- Como se mencionó anteriormente, la serie tiene mucha estacionalidad muy marcada, por lo que una red CNN pequeña tendría que hacer el trabajo sin problema. Se seleccionó para empezar 2 capas convolucionales con 16 filtros de 2 y padding same, 5 capas ocultas con relu y 50 neuronas cada una.

In [ ]:
# Creando el modelo CNN con 2 capas CNN de 16 filtros y 5 capas Dense de 50 neuronas
model_cnn, h_cnn = Utils.gen_CNN_model(X=X_train, y=y_train, val_split=0.1, n_steps=5,
                                       num_layers_cnn=2, num_filters=16, kernel_size=2, padding='same',
                                       activation='relu', num_layers_dense=5, num_neurons=50,
                                       optimizer='Adam', lr=0.003, loss='mse', metrics=['mae'],
                                       patience=20, epochs=500,  verbose=0,
                                       plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_cnn = Utils.plot_pred_test(nombre_modelo='CNN 1',
                                   title='CNN 1',
                                   pred=model_cnn.predict(X_test),
                                   test=y_test
                                   )

##### 3.2. CNN 2
- Agregando 2 capas convolucionales, quitando filtros y haciendo más grande el tamaño de los filtros. Buscamos que los filtros hagan un mejor trabajo.
- ASí mismo disminuimos el número de densasa para ver si funciona.

In [ ]:
# Creando el modelo CNN con 2 capas CNN de 16 filtros y 5 capas Dense de 50 neuronas
model_cnn2, h_cnn2 = Utils.gen_CNN_model(X=X_train, y=y_train, val_split=0.1, n_steps=5,
                                       num_layers_cnn=4, num_filters=4, kernel_size=3, padding='same',
                                       activation='relu', num_layers_dense=2, num_neurons=20,
                                       optimizer='Adam', lr=0.003, loss='mse', metrics=['mae'],
                                       patience=20, epochs=500,  verbose=0,
                                       plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_cnn2 = Utils.plot_pred_test(nombre_modelo='CNN2',
                                   title='CNN 2',
                                   pred=model_cnn2.predict(X_test),
                                   test=y_test
                                   )

##### 3.3. CNN 3
- Parece ser que si funciona, aumentaremos un poco más la parte convolucioneal y también aumentaremos la parte de predicción.
- 10 capas convolucionales con 8 filtros y tamaño de 5.
- 5 capas ocultas y 50 neuronas
- Bajando el LR para bajar en la cola un poco más e intentar llegar un poco más abajo en la función de perdida.

In [ ]:
# Creando el modelo CNN con 2 capas CNN de 16 filtros y 5 capas Dense de 50 neuronas
model_cnn3, h_cnn3 = Utils.gen_CNN_model(X=X_train, y=y_train, val_split=0.1, n_steps=5,
                                       num_layers_cnn=10, num_filters=8, kernel_size=5, padding='same',
                                       activation='relu', num_layers_dense=5, num_neurons=50,
                                       optimizer='Adam', lr=0.0003, loss='mse', metrics=['mae'],
                                       patience=20, epochs=500,  verbose=0,
                                       plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_cnn3 = Utils.plot_pred_test(nombre_modelo='CNN 3',
                                   title='CNN 3',
                                   pred=model_cnn3.predict(X_test),
                                   test=y_test
                                   )

**Conclusiones CNN**

In [ ]:
errores_CNN = Utils.concat_errores([errores_cnn, errores_cnn2, errores_cnn3])
errores_CNN

In [ ]:
errores_mejor_CNN = Utils.plot_pred_test(nombre_modelo=errores_CNN['Modelo'][0],
                                         title=errores_CNN['Modelo'][0],
                                         pred=model_cnn.predict(X_test),
                                         test=y_test)

- Podemos ver que el aumento de la parte convolucional, sin embargo al aumentarla todavía más no logramos mejorar el rendimiento, por lo que seleccioné el 2 ya que es menos complejo.

#### 4. LSTM
- Para poder usar LSTM tenemos que convertir nuestros datos a tensor, por lo que rehacemos los split y Train y test.

In [ ]:
# Separando los datos de entrenamiento y prueba y haciendolo tensor
X_tens, y_tens = Utils.split_univariate_sequence(sequence=h251,
                                                 column='H251',
                                                 n_steps=5,
                                                 tensor=True,
                                                 n_features=1)

In [ ]:
# Dividiendo los datos en train y test
X_train_tens, X_test_tens, y_train_tens, y_test_tens = Utils.split_train_test(X=X_tens,
                                                                    y=y_tens,
                                                                    train_size=0.9
                                                                    )

##### 4.1 LSTM

In [ ]:
# Creando el modelo LSTM con 2 capas LSTM de 50 neuronas y 5 capas Dense de 50 neuronas
model_lstm, h_lstm = Utils.gen_LSTM_model(X=X_train_tens, y=y_train_tens, val_split=0.1, n_steps=5, n_features=1,
                                          num_layers_lstm=2, activation_lstm='tanh', num_units_lstm=50, bidireccional=False,
                                          activation='relu', num_layers_dense=5, num_neurons=50,
                                          optimizer='Adam', lr=0.0003, loss='mse', metrics=['mae'],
                                          patience=20, epochs=500, verbose=0,
                                          plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_lstm = Utils.plot_pred_test(nombre_modelo='LSTM 1',
                                   title='LSTM 1',
                                   pred=model_lstm.predict(X_test_tens),
                                   test=y_test
                                   )

##### 4.2 LSTM
- Aumentando capas de LSTM para ver si mejora
- 4 capas densas en vez de 2

In [ ]:
# Creando el modelo LSTM con 4 capas LSTM de 50 neuronas y 5 capas Dense de 50 neuronas
model_lstm2, h_lstm2 = Utils.gen_LSTM_model(X=X_train_tens, y=y_train_tens, val_split=0.1, n_steps=5, n_features=1,
                                          num_layers_lstm=4, activation_lstm='tanh', num_units_lstm=50, bidireccional=False,
                                          activation='relu', num_layers_dense=5, num_neurons=50,
                                          optimizer='Adam', lr=0.0003, loss='mse', metrics=['mae'],
                                          patience=20, epochs=500, verbose=0,
                                          plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_lstm2 = Utils.plot_pred_test(nombre_modelo='LSTM 2',
                                   title='LSTM 2',
                                   pred=model_lstm2.predict(X_test_tens),
                                   test=y_test
                                   )

##### 4.3 LSTM

In [ ]:
# Creando el modelo LSTM con 4 capas LSTM de 50 neuronas y 5 capas Dense de 50 neuronas
model_lstm3, h_lstm3 = Utils.gen_LSTM_model(X=X_train_tens, y=y_train_tens, val_split=0.1, n_steps=5, n_features=1,
                                          num_layers_lstm=5, activation_lstm='tanh', num_units_lstm=50, bidireccional=True,
                                          activation='relu', num_layers_dense=5, num_neurons=50,
                                          optimizer='Adam', lr=0.0003, loss='mse', metrics=['mae'],
                                          patience=20, epochs=500, verbose=0,
                                          plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_lstm3 = Utils.plot_pred_test(nombre_modelo='LSTM 3',
                                   title='LSTM 3',
                                   pred=model_lstm3.predict(X_test_tens),
                                   test=y_test
                                   )

**Conclusiones**

In [ ]:
errores_LSTM = Utils.concat_errores([errores_lstm, errores_lstm2, errores_lstm3])
errores_LSTM

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
mejor_lstm = Utils.plot_pred_test(nombre_modelo=errores_LSTM['Modelo'][0],
                                  title=errores_LSTM['Modelo'][0],
                                  pred=model_lstm2.predict(X_test_tens),
                                  test=y_test)

- Tenemos un error casi nulo, sorprendemente mientras más aumentabamos las capas o la cantidad de unidades por cada (como en el caso 1 y 2 no bajo el error, parece ser que es un muy buen modelo)

#### 5. CNN - LSTM
##### 5.1. CNN - LSTM

In [ ]:
# Separando los datos de entrenamiento y prueba y haciendolo tensor con 6 pasos
X_tens2, y_tens2 = Utils.split_univariate_sequence(sequence=h229,
                                                 column='H229',
                                                 n_steps=6,
                                                 tensor=True,
                                                 n_features=1)

# Dividiendo los datos en train y test
X_train_tens2, X_test_tens2, y_train_tens2, y_test_tens2 = Utils.split_train_test(X=X_tens2,
                                                                                  y=y_tens2,
                                                                                  train_size=0.9
                                                                                    )

# Cambiando el shape del X_train_tens y X_test_tens
n_seq = 2
n_steps = 3
n_features = 1

# Cambiando el shape del X_train_tens y X_test_tens
X_train_tens2 = X_train_tens2.reshape((X_train_tens2.shape[0], n_seq, n_steps, n_features))
X_test_tens2 = X_test_tens2.reshape((X_test_tens2.shape[0], n_seq, n_steps, n_features))

In [ ]:
model_cnn_lstm1, h1_cnn_lstm = Utils.gen_CNN_LSTM_model(X=X_train_tens2, y=y_train_tens2, val_split=0.1, n_steps=5, n_features=1,
                                                  num_layers_cnn=4, num_filters=16, kernel_size=3, padding='same',
                                                  num_layers_lstm=5, activation_lstm='tanh', num_units_lstm=50,
                                                  activation='relu', num_layers_dense=5, num_neurons=50, 
                                                  optimizer='Adam', lr=0.0003, loss='mse', metrics=['mae'],
                                                  patience=20, epochs=500, verbose=0,
                                                  plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_cnn_lstm1 = Utils.plot_pred_test(nombre_modelo='CNN - LSTM 1',
                                          title='CNN - LSTM 1',
                                          pred=model_cnn_lstm1.predict(X_test_tens2),
                                          test=y_test_tens2
                                          )

##### 5.2 CNN - LSTM 2

In [ ]:
model_cnn_lstm2, h2_cnn_lstm = Utils.gen_CNN_LSTM_model(X=X_train_tens2, y=y_train_tens2, val_split=0.1, n_steps=5, n_features=1,
                                                  num_layers_cnn=2, num_filters=32, kernel_size=3, padding='same',
                                                  num_layers_lstm=5, activation_lstm='tanh', num_units_lstm=50,
                                                  activation='relu', num_layers_dense=5, num_neurons=50, 
                                                  optimizer='Adam', lr=0.0003, loss='mse', metrics=['mae'],
                                                  patience=20, epochs=500, verbose=0,
                                                  plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_cnn_lstm1 = Utils.plot_pred_test(nombre_modelo='CNN - LSTM 2',
                                          title='CNN - LSTM 2',
                                          pred=model_cnn_lstm2.predict(X_test_tens2),
                                          test=y_test_tens2
                                          )

##### 5.3 CNN - LSTM 3

In [ ]:
model_cnn_lstm3, h3_cnn_lstm = Utils.gen_CNN_LSTM_model(X=X_train_tens2, y=y_train_tens2, val_split=0.1, n_steps=5, n_features=1,
                                                  num_layers_cnn=8, num_filters=4, kernel_size=4, padding='same',
                                                  num_layers_lstm=5, activation_lstm='tanh', num_units_lstm=50,
                                                  activation='relu', num_layers_dense=5, num_neurons=50, 
                                                  optimizer='Adam', lr=0.0003, loss='mse', metrics=['mae'],
                                                  patience=20, epochs=500, verbose=0,
                                                  plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_cnn_lstm1 = Utils.plot_pred_test(nombre_modelo='CNN - LSTM 3',
                                          title='CNN - LSTM 3',
                                          pred=model_cnn_lstm3.predict(X_test_tens2),
                                          test=y_test_tens2
                                          )

**Conclusiones**

In [ ]:
errores_cnn_lstm = Utils.concat_errores([errores_cnn_lstm1, errores_cnn_lstm2, errores_cnn_lstm3])
errores_cnn_lstm

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_mejor_CNN_LSTM = Utils.plot_pred_test(nombre_modelo=errores_cnn_lstm['Modelo'][0],
                                  title=errores_cnn_lstm['Modelo'][0],
                                  pred=model_cnn_lstm1.predict(X_test_tens2),
                                  test=y_test)

**Sacando el mejor modelo**

In [ ]:
mejores_modelos = Utils.concat_errores([errores_mejor_MLP, errores_mejor_CNN, errores_mejor_lstm, errores_mejor_CNN_LSTM]).sort_values(by='MAE')
mejores_modelos

- El mejor modelo es el LSTM, por lo que es el que haremos la predicción

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_mejor_lstm = Utils.plot_pred_test(nombre_modelo=errores_LSTM['Modelo'][0],
                                  title=errores_LSTM['Modelo'][0],
                                  pred=model_lstm.predict(X_test_tens[:48]),
                                  test=y_test[:48])

- No se hizo ajuste de hiperparámetros ya que tenemos un r2 de 0.99, lo cual buscar mejorarlo es overfitearlo y desperdiciar poder de computo.

____
### 2.4. Serie H300

In [ ]:
Utils.plot_series([h300], ['H300'])

#### 1. Preprocesamiento de los datos
- Al tener una serie que es bastante Estacional (que se repite mucho los patrones cada cierto tiempo), no veo necesario hacer un procesamiento de los datos, además de que parecen ser que son bastante simétricos.

In [ ]:
# Dandole formato a la serie de tiempo
X, y = Utils.split_univariate_sequence(sequence=h300, 
                                       column='H300', 
                                       n_steps=5, 
                                       tensor=False,
                                       )

- Se seleccionó *n_steps = 5*, porque tenemos una serie de tiempo con bajadas y subidas constantes así que seleccionamos un número mediano para poder saber si es una bajada prolongada o solo es una bajada y subida y poder aprender esa estacionalidad de la serie.

In [ ]:
# Separando los datos de entrenamiento y prueba
X_train, X_test, y_train, y_test = Utils.split_train_test(X=X, 
                                                          y=y, 
                                                          train_size=0.9)

#### 2. MLP - Multy Layer Perceptron
##### 2.1 MLP
- 2 Capas ocultas con 20 neuronas

In [ ]:
# Creando el modelo MLP con 5 capas ocultas de 50 neuronas
model_mlp1, h_mlp1 = Utils.gen_MLP_model(X=X_train, y=y_train, val_split=0.1, n_steps=5,
                                         activation='relu', num_layers=2, num_neurons=20,
                                         optimizer='Adam', lr=0.0001, loss='mse', metrics=['mae'],
                                         patience=20, epochs=500,  verbose=0,
                                         plot_history=True)


In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_mlp1 = Utils.plot_pred_test(nombre_modelo='MLP 1',
                                   title='MLP',
                                   pred=model_mlp1.predict(X_test),
                                   test=y_test
                                   )

##### 2.2 MLP
- 4 Capas ocultas con 30 neuronas

In [ ]:
# Creando el modelo MLP con 4 capas ocultas de 30 neuronas
model_mlp2, h_mlp2 = Utils.gen_MLP_model(X=X_train, y=y_train, val_split=0.1, n_steps=5,
                                         activation='relu', num_layers=4, num_neurons=30,
                                         optimizer='Adam', lr=0.003, loss='mse', metrics=['mae'],
                                         patience=20, epochs=500,  verbose=0,
                                         plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_mlp2 = Utils.plot_pred_test(nombre_modelo='MLP 2',
                                   title='MLP',
                                   pred=model_mlp2.predict(X_test),
                                   test=y_test
                                   )

##### 2.3 MLP
- 5 Capas ocultas con 50 neuronas

In [ ]:
# Creando el modelo MLP con 4 capas ocultas de 30 neuronas
model_mlp3, h_mlp3 = Utils.gen_MLP_model(X=X_train, y=y_train, val_split=0.1, n_steps=5,
                                         activation='relu', num_layers=5, num_neurons=50,
                                         optimizer='Adam', lr=0.003, loss='mse', metrics=['mae'],
                                         patience=20, epochs=500,  verbose=0,
                                         plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_mlp3 = Utils.plot_pred_test(nombre_modelo='MLP 3',
                                   title='MLP',
                                   pred=model_mlp3.predict(X_test),
                                   test=y_test
                                   )

**Conclusiones MLP**

In [ ]:
errores_MLP = Utils.concat_errores([errores_mlp1, errores_mlp2, errores_mlp3])
errores_MLP

In [ ]:
errores_mejor_MLP = Utils.plot_pred_test(nombre_modelo=errores_MLP['Modelo'][0],
                                         title=errores_MLP['Modelo'][0],
                                         pred=model_mlp1.predict(X_test),
                                         test=y_test)

#### 3. CNN - Convolutional Neural Network
##### 3.1. CNN 1
- Como se mencionó anteriormente, la serie tiene mucha estacionalidad muy marcada, por lo que una red CNN pequeña tendría que hacer el trabajo sin problema. Se seleccionó para empezar 2 capas convolucionales con 16 filtros de 2 y padding same, 5 capas ocultas con relu y 50 neuronas cada una.

In [ ]:
# Creando el modelo CNN con 2 capas CNN de 16 filtros y 5 capas Dense de 50 neuronas
model_cnn, h_cnn = Utils.gen_CNN_model(X=X_train, y=y_train, val_split=0.1, n_steps=5,
                                       num_layers_cnn=2, num_filters=16, kernel_size=2, padding='same',
                                       activation='relu', num_layers_dense=5, num_neurons=50,
                                       optimizer='Adam', lr=0.003, loss='mse', metrics=['mae'],
                                       patience=20, epochs=500,  verbose=0,
                                       plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_cnn = Utils.plot_pred_test(nombre_modelo='CNN 1',
                                   title='CNN 1',
                                   pred=model_cnn.predict(X_test),
                                   test=y_test
                                   )

##### 3.2. CNN 2
- Agregando 2 capas convolucionales, quitando filtros y haciendo más grande el tamaño de los filtros. Buscamos que los filtros hagan un mejor trabajo.
- ASí mismo disminuimos el número de densasa para ver si funciona.

In [ ]:
# Creando el modelo CNN con 2 capas CNN de 16 filtros y 5 capas Dense de 50 neuronas
model_cnn2, h_cnn2 = Utils.gen_CNN_model(X=X_train, y=y_train, val_split=0.1, n_steps=5,
                                       num_layers_cnn=4, num_filters=4, kernel_size=3, padding='same',
                                       activation='relu', num_layers_dense=2, num_neurons=20,
                                       optimizer='Adam', lr=0.003, loss='mse', metrics=['mae'],
                                       patience=20, epochs=500,  verbose=0,
                                       plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_cnn2 = Utils.plot_pred_test(nombre_modelo='CNN2',
                                   title='CNN 2',
                                   pred=model_cnn2.predict(X_test),
                                   test=y_test
                                   )

##### 3.3. CNN 3
- Parece ser que si funciona, aumentaremos un poco más la parte convolucioneal y también aumentaremos la parte de predicción.
- 10 capas convolucionales con 8 filtros y tamaño de 5.
- 5 capas ocultas y 50 neuronas
- Bajando el LR para bajar en la cola un poco más e intentar llegar un poco más abajo en la función de perdida.

In [ ]:
# Creando el modelo CNN con 2 capas CNN de 16 filtros y 5 capas Dense de 50 neuronas
model_cnn3, h_cnn3 = Utils.gen_CNN_model(X=X_train, y=y_train, val_split=0.1, n_steps=5,
                                       num_layers_cnn=10, num_filters=8, kernel_size=5, padding='same',
                                       activation='relu', num_layers_dense=5, num_neurons=50,
                                       optimizer='Adam', lr=0.0003, loss='mse', metrics=['mae'],
                                       patience=20, epochs=500,  verbose=0,
                                       plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_cnn3 = Utils.plot_pred_test(nombre_modelo='CNN 3',
                                   title='CNN 3',
                                   pred=model_cnn3.predict(X_test),
                                   test=y_test
                                   )

**Conclusiones CNN**

In [ ]:
errores_CNN = Utils.concat_errores([errores_cnn, errores_cnn2, errores_cnn3])
errores_CNN

In [ ]:
errores_mejor_CNN = Utils.plot_pred_test(nombre_modelo=errores_CNN['Modelo'][0],
                                         title=errores_CNN['Modelo'][0],
                                         pred=model_cnn.predict(X_test),
                                         test=y_test)

- Podemos ver que el aumento de la parte convolucional, sin embargo al aumentarla todavía más no logramos mejorar el rendimiento, por lo que seleccioné el 2 ya que es menos complejo.

#### 4. LSTM
- Para poder usar LSTM tenemos que convertir nuestros datos a tensor, por lo que rehacemos los split y Train y test.

In [ ]:
# Separando los datos de entrenamiento y prueba y haciendolo tensor
X_tens, y_tens = Utils.split_univariate_sequence(sequence=h300,
                                                 column='H300',
                                                 n_steps=5,
                                                 tensor=True,
                                                 n_features=1)

In [ ]:
# Dividiendo los datos en train y test
X_train_tens, X_test_tens, y_train_tens, y_test_tens = Utils.split_train_test(X=X_tens,
                                                                    y=y_tens,
                                                                    train_size=0.9
                                                                    )

##### 4.1 LSTM

In [ ]:
# Creando el modelo LSTM con 2 capas LSTM de 50 neuronas y 5 capas Dense de 50 neuronas
model_lstm, h_lstm = Utils.gen_LSTM_model(X=X_train_tens, y=y_train_tens, val_split=0.1, n_steps=5, n_features=1,
                                          num_layers_lstm=2, activation_lstm='tanh', num_units_lstm=50, bidireccional=False,
                                          activation='relu', num_layers_dense=5, num_neurons=50,
                                          optimizer='Adam', lr=0.0003, loss='mse', metrics=['mae'],
                                          patience=20, epochs=500, verbose=0,
                                          plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_lstm = Utils.plot_pred_test(nombre_modelo='LSTM 1',
                                   title='LSTM 1',
                                   pred=model_lstm.predict(X_test_tens),
                                   test=y_test
                                   )

##### 4.2 LSTM
- Aumentando capas de LSTM para ver si mejora
- 4 capas densas en vez de 2

In [ ]:
# Creando el modelo LSTM con 4 capas LSTM de 50 neuronas y 5 capas Dense de 50 neuronas
model_lstm2, h_lstm2 = Utils.gen_LSTM_model(X=X_train_tens, y=y_train_tens, val_split=0.1, n_steps=5, n_features=1,
                                          num_layers_lstm=4, activation_lstm='tanh', num_units_lstm=50, bidireccional=False,
                                          activation='relu', num_layers_dense=5, num_neurons=50,
                                          optimizer='Adam', lr=0.0003, loss='mse', metrics=['mae'],
                                          patience=20, epochs=500, verbose=0,
                                          plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_lstm2 = Utils.plot_pred_test(nombre_modelo='LSTM 2',
                                   title='LSTM 2',
                                   pred=model_lstm2.predict(X_test_tens),
                                   test=y_test
                                   )

##### 4.3 LSTM

In [ ]:
# Creando el modelo LSTM con 4 capas LSTM de 50 neuronas y 5 capas Dense de 50 neuronas
model_lstm3, h_lstm3 = Utils.gen_LSTM_model(X=X_train_tens, y=y_train_tens, val_split=0.1, n_steps=5, n_features=1,
                                          num_layers_lstm=5, activation_lstm='tanh', num_units_lstm=50, bidireccional=True,
                                          activation='relu', num_layers_dense=5, num_neurons=50,
                                          optimizer='Adam', lr=0.0003, loss='mse', metrics=['mae'],
                                          patience=20, epochs=500, verbose=0,
                                          plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_lstm3 = Utils.plot_pred_test(nombre_modelo='LSTM 3',
                                   title='LSTM 3',
                                   pred=model_lstm3.predict(X_test_tens),
                                   test=y_test
                                   )

**Conclusiones**

In [ ]:
errores_LSTM = Utils.concat_errores([errores_lstm, errores_lstm2, errores_lstm3])
errores_LSTM

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
mejor_lstm = Utils.plot_pred_test(nombre_modelo=errores_LSTM['Modelo'][0],
                                  title=errores_LSTM['Modelo'][0],
                                  pred=model_lstm2.predict(X_test_tens),
                                  test=y_test)

- Tenemos un error casi nulo, sorprendemente mientras más aumentabamos las capas o la cantidad de unidades por cada (como en el caso 1 y 2 no bajo el error, parece ser que es un muy buen modelo)

#### 5. CNN - LSTM
##### 5.1. CNN - LSTM

In [ ]:
# Separando los datos de entrenamiento y prueba y haciendolo tensor con 6 pasos
X_tens2, y_tens2 = Utils.split_univariate_sequence(sequence=h229,
                                                 column='H229',
                                                 n_steps=6,
                                                 tensor=True,
                                                 n_features=1)

# Dividiendo los datos en train y test
X_train_tens2, X_test_tens2, y_train_tens2, y_test_tens2 = Utils.split_train_test(X=X_tens2,
                                                                                  y=y_tens2,
                                                                                  train_size=0.9
                                                                                    )

# Cambiando el shape del X_train_tens y X_test_tens
n_seq = 2
n_steps = 3
n_features = 1

# Cambiando el shape del X_train_tens y X_test_tens
X_train_tens2 = X_train_tens2.reshape((X_train_tens2.shape[0], n_seq, n_steps, n_features))
X_test_tens2 = X_test_tens2.reshape((X_test_tens2.shape[0], n_seq, n_steps, n_features))

In [ ]:
model_cnn_lstm1, h1_cnn_lstm = Utils.gen_CNN_LSTM_model(X=X_train_tens2, y=y_train_tens2, val_split=0.1, n_steps=5, n_features=1,
                                                  num_layers_cnn=4, num_filters=16, kernel_size=3, padding='same',
                                                  num_layers_lstm=5, activation_lstm='tanh', num_units_lstm=50,
                                                  activation='relu', num_layers_dense=5, num_neurons=50, 
                                                  optimizer='Adam', lr=0.0003, loss='mse', metrics=['mae'],
                                                  patience=20, epochs=500, verbose=0,
                                                  plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_cnn_lstm1 = Utils.plot_pred_test(nombre_modelo='CNN - LSTM 1',
                                          title='CNN - LSTM 1',
                                          pred=model_cnn_lstm1.predict(X_test_tens2),
                                          test=y_test_tens2
                                          )

##### 5.2 CNN - LSTM 2

In [ ]:
model_cnn_lstm2, h2_cnn_lstm = Utils.gen_CNN_LSTM_model(X=X_train_tens2, y=y_train_tens2, val_split=0.1, n_steps=5, n_features=1,
                                                  num_layers_cnn=2, num_filters=32, kernel_size=3, padding='same',
                                                  num_layers_lstm=5, activation_lstm='tanh', num_units_lstm=50,
                                                  activation='relu', num_layers_dense=5, num_neurons=50, 
                                                  optimizer='Adam', lr=0.0003, loss='mse', metrics=['mae'],
                                                  patience=20, epochs=500, verbose=0,
                                                  plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_cnn_lstm1 = Utils.plot_pred_test(nombre_modelo='CNN - LSTM 2',
                                          title='CNN - LSTM 2',
                                          pred=model_cnn_lstm2.predict(X_test_tens2),
                                          test=y_test_tens2
                                          )

##### 5.3 CNN - LSTM 3

In [ ]:
model_cnn_lstm3, h3_cnn_lstm = Utils.gen_CNN_LSTM_model(X=X_train_tens2, y=y_train_tens2, val_split=0.1, n_steps=5, n_features=1,
                                                  num_layers_cnn=8, num_filters=4, kernel_size=4, padding='same',
                                                  num_layers_lstm=5, activation_lstm='tanh', num_units_lstm=50,
                                                  activation='relu', num_layers_dense=5, num_neurons=50, 
                                                  optimizer='Adam', lr=0.0003, loss='mse', metrics=['mae'],
                                                  patience=20, epochs=500, verbose=0,
                                                  plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_cnn_lstm1 = Utils.plot_pred_test(nombre_modelo='CNN - LSTM 3',
                                          title='CNN - LSTM 3',
                                          pred=model_cnn_lstm3.predict(X_test_tens2),
                                          test=y_test_tens2
                                          )

**Conclusiones**

In [ ]:
errores_cnn_lstm = Utils.concat_errores([errores_cnn_lstm1, errores_cnn_lstm2, errores_cnn_lstm3])
errores_cnn_lstm

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_mejor_CNN_LSTM = Utils.plot_pred_test(nombre_modelo=errores_cnn_lstm['Modelo'][0],
                                  title=errores_cnn_lstm['Modelo'][0],
                                  pred=model_cnn_lstm1.predict(X_test_tens2),
                                  test=y_test)

**Sacando el mejor modelo**

In [ ]:
mejores_modelos = Utils.concat_errores([errores_mejor_MLP, errores_mejor_CNN, errores_mejor_lstm, errores_mejor_CNN_LSTM]).sort_values(by='MAE')
mejores_modelos

- El mejor modelo es el LSTM, por lo que es el que haremos la predicción

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_mejor_lstm = Utils.plot_pred_test(nombre_modelo=errores_LSTM['Modelo'][0],
                                  title=errores_LSTM['Modelo'][0],
                                  pred=model_lstm.predict(X_test_tens[:48]),
                                  test=y_test[:48])

- No se hizo ajuste de hiperparámetros ya que tenemos un r2 de 0.99, lo cual buscar mejorarlo es overfitearlo y desperdiciar poder de computo.

____
### 2.5. Serie H405

In [ ]:
Utils.plot_series([h405], ['H405'])

#### 1. Preprocesamiento de los datos
- Al tener una serie que es bastante Estacional (que se repite mucho los patrones cada cierto tiempo), no veo necesario hacer un procesamiento de los datos, además de que parecen ser que son bastante simétricos.

In [ ]:
# Dandole formato a la serie de tiempo
X, y = Utils.split_univariate_sequence(sequence=h405, 
                                       column='H405', 
                                       n_steps=5, 
                                       tensor=False,
                                       )

- Se seleccionó *n_steps = 5*, porque tenemos una serie de tiempo con bajadas y subidas constantes así que seleccionamos un número mediano para poder saber si es una bajada prolongada o solo es una bajada y subida y poder aprender esa estacionalidad de la serie.

In [ ]:
# Separando los datos de entrenamiento y prueba
X_train, X_test, y_train, y_test = Utils.split_train_test(X=X, 
                                                          y=y, 
                                                          train_size=0.9)

#### 2. MLP - Multy Layer Perceptron
##### 2.1 MLP
- 2 Capas ocultas con 20 neuronas

In [ ]:
# Creando el modelo MLP con 5 capas ocultas de 50 neuronas
model_mlp1, h_mlp1 = Utils.gen_MLP_model(X=X_train, y=y_train, val_split=0.1, n_steps=5,
                                         activation='relu', num_layers=2, num_neurons=20,
                                         optimizer='Adam', lr=0.0001, loss='mse', metrics=['mae'],
                                         patience=20, epochs=500,  verbose=0,
                                         plot_history=True)


In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_mlp1 = Utils.plot_pred_test(nombre_modelo='MLP 1',
                                   title='MLP',
                                   pred=model_mlp1.predict(X_test),
                                   test=y_test
                                   )

##### 2.2 MLP
- 4 Capas ocultas con 30 neuronas

In [ ]:
# Creando el modelo MLP con 4 capas ocultas de 30 neuronas
model_mlp2, h_mlp2 = Utils.gen_MLP_model(X=X_train, y=y_train, val_split=0.1, n_steps=5,
                                         activation='relu', num_layers=4, num_neurons=30,
                                         optimizer='Adam', lr=0.003, loss='mse', metrics=['mae'],
                                         patience=20, epochs=500,  verbose=0,
                                         plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_mlp2 = Utils.plot_pred_test(nombre_modelo='MLP 2',
                                   title='MLP',
                                   pred=model_mlp2.predict(X_test),
                                   test=y_test
                                   )

##### 2.3 MLP
- 5 Capas ocultas con 50 neuronas

In [ ]:
# Creando el modelo MLP con 4 capas ocultas de 30 neuronas
model_mlp3, h_mlp3 = Utils.gen_MLP_model(X=X_train, y=y_train, val_split=0.1, n_steps=5,
                                         activation='relu', num_layers=5, num_neurons=50,
                                         optimizer='Adam', lr=0.003, loss='mse', metrics=['mae'],
                                         patience=20, epochs=500,  verbose=0,
                                         plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_mlp3 = Utils.plot_pred_test(nombre_modelo='MLP 3',
                                   title='MLP',
                                   pred=model_mlp3.predict(X_test),
                                   test=y_test
                                   )

**Conclusiones MLP**

In [ ]:
errores_MLP = Utils.concat_errores([errores_mlp1, errores_mlp2, errores_mlp3])
errores_MLP

In [ ]:
errores_mejor_MLP = Utils.plot_pred_test(nombre_modelo=errores_MLP['Modelo'][0],
                                         title=errores_MLP['Modelo'][0],
                                         pred=model_mlp2.predict(X_test),
                                         test=y_test)

#### 3. CNN - Convolutional Neural Network
##### 3.1. CNN 1
- Como se mencionó anteriormente, la serie tiene mucha estacionalidad muy marcada, por lo que una red CNN pequeña tendría que hacer el trabajo sin problema. Se seleccionó para empezar 2 capas convolucionales con 16 filtros de 2 y padding same, 5 capas ocultas con relu y 50 neuronas cada una.

In [ ]:
# Creando el modelo CNN con 2 capas CNN de 16 filtros y 5 capas Dense de 50 neuronas
model_cnn, h_cnn = Utils.gen_CNN_model(X=X_train, y=y_train, val_split=0.1, n_steps=5,
                                       num_layers_cnn=2, num_filters=16, kernel_size=2, padding='same',
                                       activation='relu', num_layers_dense=5, num_neurons=50,
                                       optimizer='Adam', lr=0.003, loss='mse', metrics=['mae'],
                                       patience=20, epochs=500,  verbose=0,
                                       plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_cnn = Utils.plot_pred_test(nombre_modelo='CNN 1',
                                   title='CNN 1',
                                   pred=model_cnn.predict(X_test),
                                   test=y_test
                                   )

##### 3.2. CNN 2
- Agregando 2 capas convolucionales, quitando filtros y haciendo más grande el tamaño de los filtros. Buscamos que los filtros hagan un mejor trabajo.
- ASí mismo disminuimos el número de densasa para ver si funciona.

In [ ]:
# Creando el modelo CNN con 2 capas CNN de 16 filtros y 5 capas Dense de 50 neuronas
model_cnn2, h_cnn2 = Utils.gen_CNN_model(X=X_train, y=y_train, val_split=0.1, n_steps=5,
                                       num_layers_cnn=4, num_filters=4, kernel_size=3, padding='same',
                                       activation='relu', num_layers_dense=2, num_neurons=20,
                                       optimizer='Adam', lr=0.003, loss='mse', metrics=['mae'],
                                       patience=20, epochs=500,  verbose=0,
                                       plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_cnn2 = Utils.plot_pred_test(nombre_modelo='CNN2',
                                   title='CNN 2',
                                   pred=model_cnn2.predict(X_test),
                                   test=y_test
                                   )

##### 3.3. CNN 3
- Parece ser que si funciona, aumentaremos un poco más la parte convolucioneal y también aumentaremos la parte de predicción.
- 10 capas convolucionales con 8 filtros y tamaño de 5.
- 5 capas ocultas y 50 neuronas
- Bajando el LR para bajar en la cola un poco más e intentar llegar un poco más abajo en la función de perdida.

In [ ]:
# Creando el modelo CNN con 2 capas CNN de 16 filtros y 5 capas Dense de 50 neuronas
model_cnn3, h_cnn3 = Utils.gen_CNN_model(X=X_train, y=y_train, val_split=0.1, n_steps=5,
                                       num_layers_cnn=10, num_filters=8, kernel_size=5, padding='same',
                                       activation='relu', num_layers_dense=5, num_neurons=50,
                                       optimizer='Adam', lr=0.0003, loss='mse', metrics=['mae'],
                                       patience=20, epochs=500,  verbose=0,
                                       plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_cnn3 = Utils.plot_pred_test(nombre_modelo='CNN 3',
                                   title='CNN 3',
                                   pred=model_cnn3.predict(X_test),
                                   test=y_test
                                   )

**Conclusiones CNN**

In [ ]:
errores_CNN = Utils.concat_errores([errores_cnn, errores_cnn2, errores_cnn3])
errores_CNN

In [ ]:
errores_mejor_CNN = Utils.plot_pred_test(nombre_modelo=errores_CNN['Modelo'][0],
                                         title=errores_CNN['Modelo'][0],
                                         pred=model_cnn.predict(X_test),
                                         test=y_test)

- Podemos ver que el aumento de la parte convolucional, sin embargo al aumentarla todavía más no logramos mejorar el rendimiento, por lo que seleccioné el 2 ya que es menos complejo.

#### 4. LSTM
- Para poder usar LSTM tenemos que convertir nuestros datos a tensor, por lo que rehacemos los split y Train y test.

In [ ]:
# Separando los datos de entrenamiento y prueba y haciendolo tensor
X_tens, y_tens = Utils.split_univariate_sequence(sequence=h405,
                                                 column='H405',
                                                 n_steps=5,
                                                 tensor=True,
                                                 n_features=1)

In [ ]:
# Dividiendo los datos en train y test
X_train_tens, X_test_tens, y_train_tens, y_test_tens = Utils.split_train_test(X=X_tens,
                                                                    y=y_tens,
                                                                    train_size=0.9
                                                                    )

##### 4.1 LSTM

In [ ]:
# Creando el modelo LSTM con 2 capas LSTM de 50 neuronas y 5 capas Dense de 50 neuronas
model_lstm, h_lstm = Utils.gen_LSTM_model(X=X_train_tens, y=y_train_tens, val_split=0.1, n_steps=5, n_features=1,
                                          num_layers_lstm=2, activation_lstm='tanh', num_units_lstm=50, bidireccional=False,
                                          activation='relu', num_layers_dense=5, num_neurons=50,
                                          optimizer='Adam', lr=0.0003, loss='mse', metrics=['mae'],
                                          patience=20, epochs=500, verbose=0,
                                          plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_lstm = Utils.plot_pred_test(nombre_modelo='LSTM 1',
                                   title='LSTM 1',
                                   pred=model_lstm.predict(X_test_tens),
                                   test=y_test
                                   )

##### 4.2 LSTM
- Aumentando capas de LSTM para ver si mejora
- 4 capas densas en vez de 2

In [ ]:
# Creando el modelo LSTM con 4 capas LSTM de 50 neuronas y 5 capas Dense de 50 neuronas
model_lstm2, h_lstm2 = Utils.gen_LSTM_model(X=X_train_tens, y=y_train_tens, val_split=0.1, n_steps=5, n_features=1,
                                          num_layers_lstm=4, activation_lstm='tanh', num_units_lstm=50, bidireccional=False,
                                          activation='relu', num_layers_dense=5, num_neurons=50,
                                          optimizer='Adam', lr=0.0003, loss='mse', metrics=['mae'],
                                          patience=20, epochs=500, verbose=0,
                                          plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_lstm2 = Utils.plot_pred_test(nombre_modelo='LSTM 2',
                                   title='LSTM 2',
                                   pred=model_lstm2.predict(X_test),
                                   test=y_test
                                   )

##### 4.3 LSTM

In [ ]:
# Creando el modelo LSTM con 4 capas LSTM de 50 neuronas y 5 capas Dense de 50 neuronas
model_lstm3, h_lstm3 = Utils.gen_LSTM_model(X=X_train_tens, y=y_train_tens, val_split=0.1, n_steps=5, n_features=1,
                                          num_layers_lstm=5, activation_lstm='tanh', num_units_lstm=50, bidireccional=True,
                                          activation='relu', num_layers_dense=5, num_neurons=50,
                                          optimizer='Adam', lr=0.0003, loss='mse', metrics=['mae'],
                                          patience=20, epochs=500, verbose=0,
                                          plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_lstm3 = Utils.plot_pred_test(nombre_modelo='LSTM 3',
                                   title='LSTM 3',
                                   pred=model_lstm3.predict(X_test),
                                   test=y_test
                                   )

**Conclusiones**

In [ ]:
errores_LSTM = Utils.concat_errores([errores_lstm, errores_lstm2, errores_lstm3])
errores_LSTM

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
mejor_lstm = Utils.plot_pred_test(nombre_modelo=errores_LSTM['Modelo'][0],
                                  title=errores_LSTM['Modelo'][0],
                                  pred=model_lstm.predict(X_test_tens),
                                  test=y_test)

- Tenemos un error casi nulo, sorprendemente mientras más aumentabamos las capas o la cantidad de unidades por cada (como en el caso 1 y 2 no bajo el error, parece ser que es un muy buen modelo)

#### 5. CNN - LSTM
##### 5.1. CNN - LSTM

In [ ]:
# Separando los datos de entrenamiento y prueba y haciendolo tensor con 6 pasos
X_tens2, y_tens2 = Utils.split_univariate_sequence(sequence=h229,
                                                 column='H229',
                                                 n_steps=6,
                                                 tensor=True,
                                                 n_features=1)

# Dividiendo los datos en train y test
X_train_tens2, X_test_tens2, y_train_tens2, y_test_tens2 = Utils.split_train_test(X=X_tens2,
                                                                                  y=y_tens2,
                                                                                  train_size=0.9
                                                                                    )

# Cambiando el shape del X_train_tens y X_test_tens
n_seq = 2
n_steps = 3
n_features = 1

# Cambiando el shape del X_train_tens y X_test_tens
X_train_tens2 = X_train_tens2.reshape((X_train_tens2.shape[0], n_seq, n_steps, n_features))
X_test_tens2 = X_test_tens2.reshape((X_test_tens2.shape[0], n_seq, n_steps, n_features))

In [ ]:
model_cnn_lstm1, h1_cnn_lstm = Utils.gen_CNN_LSTM_model(X=X_train_tens2, y=y_train_tens2, val_split=0.1, n_steps=5, n_features=1,
                                                  num_layers_cnn=4, num_filters=16, kernel_size=3, padding='same',
                                                  num_layers_lstm=5, activation_lstm='tanh', num_units_lstm=50,
                                                  activation='relu', num_layers_dense=5, num_neurons=50, 
                                                  optimizer='Adam', lr=0.0003, loss='mse', metrics=['mae'],
                                                  patience=20, epochs=500, verbose=0,
                                                  plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_cnn_lstm1 = Utils.plot_pred_test(nombre_modelo='CNN - LSTM 1',
                                          title='CNN - LSTM 1',
                                          pred=model_cnn_lstm1.predict(X_test_tens2),
                                          test=y_test_tens2
                                          )

##### 5.2 CNN - LSTM 2

In [ ]:
model_cnn_lstm2, h2_cnn_lstm = Utils.gen_CNN_LSTM_model(X=X_train_tens2, y=y_train_tens2, val_split=0.1, n_steps=5, n_features=1,
                                                  num_layers_cnn=2, num_filters=32, kernel_size=3, padding='same',
                                                  num_layers_lstm=5, activation_lstm='tanh', num_units_lstm=50,
                                                  activation='relu', num_layers_dense=5, num_neurons=50, 
                                                  optimizer='Adam', lr=0.0003, loss='mse', metrics=['mae'],
                                                  patience=20, epochs=500, verbose=0,
                                                  plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_cnn_lstm1 = Utils.plot_pred_test(nombre_modelo='CNN - LSTM 2',
                                          title='CNN - LSTM 2',
                                          pred=model_cnn_lstm2.predict(X_test_tens2),
                                          test=y_test_tens2
                                          )

##### 5.3 CNN - LSTM 3

In [ ]:
model_cnn_lstm3, h3_cnn_lstm = Utils.gen_CNN_LSTM_model(X=X_train_tens2, y=y_train_tens2, val_split=0.1, n_steps=5, n_features=1,
                                                  num_layers_cnn=8, num_filters=4, kernel_size=4, padding='same',
                                                  num_layers_lstm=5, activation_lstm='tanh', num_units_lstm=50,
                                                  activation='relu', num_layers_dense=5, num_neurons=50, 
                                                  optimizer='Adam', lr=0.0003, loss='mse', metrics=['mae'],
                                                  patience=20, epochs=500, verbose=0,
                                                  plot_history=True)

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_cnn_lstm1 = Utils.plot_pred_test(nombre_modelo='CNN - LSTM 3',
                                          title='CNN - LSTM 3',
                                          pred=model_cnn_lstm3.predict(X_test_tens2),
                                          test=y_test_tens2
                                          )

**Conclusiones**

In [ ]:
errores_cnn_lstm = Utils.concat_errores([errores_cnn_lstm1, errores_cnn_lstm2, errores_cnn_lstm3])
errores_cnn_lstm

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_mejor_CNN_LSTM = Utils.plot_pred_test(nombre_modelo=errores_cnn_lstm['Modelo'][0],
                                  title=errores_cnn_lstm['Modelo'][0],
                                  pred=model_cnn_lstm1.predict(X_test_tens2),
                                  test=y_test)

**Sacando el mejor modelo**

In [ ]:
mejores_modelos = Utils.concat_errores([errores_mejor_MLP, errores_mejor_CNN, errores_mejor_lstm, errores_mejor_CNN_LSTM]).sort_values(by='MAE')
mejores_modelos

- El mejor modelo es el LSTM, por lo que es el que haremos la predicción

In [ ]:
# Obteniendo los errores y ploteando predicciones vs test
errores_mejor_lstm = Utils.plot_pred_test(nombre_modelo=errores_LSTM['Modelo'][0],
                                  title=errores_LSTM['Modelo'][0],
                                  pred=model_lstm.predict(X_test_tens[:48]),
                                  test=y_test[:48])

- No se hizo ajuste de hiperparámetros ya que tenemos un r2 de 0.99, lo cual buscar mejorarlo es overfitearlo y desperdiciar poder de computo.